In [13]:
from ultralytics.utils.custom_utils.init_setup import setup, delete_all_fiftyone_datasets

delete_all_fiftyone_datasets()
setup(0)

Loading dataset
 100% |███████████████| 6732/6732 [6.0s elapsed, 0s remaining, 1.1K samples/s]      
 100% |█████████████████| 936/936 [954.7ms elapsed, 0s remaining, 981.3 samples/s]      
 100% |███████████████| 1889/1889 [2.6s elapsed, 0s remaining, 733.9 samples/s]      
Adding transects


9557it [00:06, 1368.82it/s]


Adding dataset attributes


100%|██████████| 9557/9557 [00:11<00:00, 866.16it/s] 


In [18]:
from ultralytics.utils.custom_utils.helpers import get_fiftyone_dataset
import fiftyone as fo

# dataset, classes = get_fiftyone_dataset(0)

# sesh = fo.launch_app(dataset)

In [15]:
import os

def create_missing_label_files(root_dir):
    # Define the root directory
    data_dir = os.path.join(root_dir, 'data', 'data')

    # Iterate through train, valid, and test folders
    for split in ['train', 'valid', 'test']:
        split_dir = os.path.join(data_dir, split)

        # Iterate through images folder in each split
        images_dir = os.path.join(split_dir, 'images')
        for image_file in os.listdir(images_dir):
            image_name, image_ext = os.path.splitext(image_file)
            label_file = image_name + '.txt'
            label_path = os.path.join(split_dir, 'labels', label_file)

            # Check if the label file exists, if not create an empty one
            if not os.path.exists(label_path):
                with open(label_path, 'w') as f:
                    pass  # Create an empty file

# Example usage
create_missing_label_files('/Users/sigurdbakkerud/Projects/master-thesis/ultralytics/')


In [28]:
import os

def convert_yolo_to_bbox(yolo_annotation):
    # Parse the YOLO annotation string
    parts = yolo_annotation.split()
    class_id = int(parts[0])
    coordinates = list(map(float, parts[1:]))

    # Extract x and y coordinates
    x_coords = coordinates[::2]
    y_coords = coordinates[1::2]

    # Calculate bounding box coordinates
    x_min = min(x_coords)
    x_max = max(x_coords)
    y_min = min(y_coords)
    y_max = max(y_coords)

    # Calculate box width and height
    width = x_max - x_min
    height = y_max - y_min

    # Calculate box center
    x_center = x_min + width / 2
    y_center = y_min + height / 2

    return class_id, x_center, y_center, width, height

def convert_annotations(folder):
    for subdir, dirs, files in os.walk(folder):
        if subdir == "data":
            for file in files:
                if file.endswith(".txt"):
                    file_path = os.path.join(subdir, file)
                    with open(file_path, 'r') as f:
                        lines = f.readlines()
                    with open(file_path, 'w') as f:
                        for line in lines:
                            if len(line.split() > 5): # Must be mask if more than 5 numbers in line
                                class_id, x_center, y_center, width, height = convert_yolo_to_bbox(line)
                                bbox_line = f"{class_id} {x_center} {y_center} {width} {height}\n"
                                f.write(bbox_line)

# Change the directory accordingly
convert_annotations('/Users/sigurdbakkerud/Projects/master-thesis/ultralytics/data/')


/Users/sigurdbakkerud/Projects/master-thesis/ultralytics/data/
/Users/sigurdbakkerud/Projects/master-thesis/ultralytics/data/Brackish Underwater
/Users/sigurdbakkerud/Projects/master-thesis/ultralytics/data/Brackish Underwater/valid
/Users/sigurdbakkerud/Projects/master-thesis/ultralytics/data/Brackish Underwater/valid/images
/Users/sigurdbakkerud/Projects/master-thesis/ultralytics/data/Brackish Underwater/valid/labels
/Users/sigurdbakkerud/Projects/master-thesis/ultralytics/data/Brackish Underwater/test
/Users/sigurdbakkerud/Projects/master-thesis/ultralytics/data/Brackish Underwater/test/images
/Users/sigurdbakkerud/Projects/master-thesis/ultralytics/data/Brackish Underwater/test/labels
/Users/sigurdbakkerud/Projects/master-thesis/ultralytics/data/Brackish Underwater/train
/Users/sigurdbakkerud/Projects/master-thesis/ultralytics/data/Brackish Underwater/train/images
/Users/sigurdbakkerud/Projects/master-thesis/ultralytics/data/Brackish Underwater/train/labels
/Users/sigurdbakkerud/Pr

In [7]:
from ultralytics.config import get_yolo_classes

print(get_yolo_classes())

[3, 5, 9, 10, 11, 15, 16, 17, 19, 20, 21, 22, 23, 28, 32]
